In [1]:
import pandas as pd
import re
import time
import numpy as np
import math
import gc
import pickle
import os
from sklearn.metrics import roc_auc_score,log_loss,f1_score
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import jieba as jb
import time

df_train_1 = pd.read_csv("../data/new/train_1.csv",parse_dates=["时间"])
df_train_2 = pd.read_csv("../data/new/train_2.csv",parse_dates=["时间"])
df_train_3 = pd.read_csv("../data/new/train_3.csv",parse_dates=["时间"])
df_train_4 = pd.read_csv("../data/new/train_4.csv",parse_dates=["时间"])

df_test_1 = pd.read_csv("../data/new/test_1.csv",parse_dates=["时间"])
df_test_2 = pd.read_csv("../data/new/test_2.csv",parse_dates=["时间"])
df_test_3 = pd.read_csv("../data/new/test_3.csv",parse_dates=["时间"])
df_test_4 = pd.read_csv("../data/new/test_4.csv",parse_dates=["时间"])

df_train_1["type"] = 1
df_train_2["type"] = 2
df_train_3["type"] = 3
df_train_4["type"] = 4
df_test_1["type"] = 1
df_test_2["type"] = 2
df_test_3["type"] = 3
df_test_4["type"] = 4

df_train_1["fea_1"] = 10*0.03
df_train_2["fea_1"] = 10*0.03
df_train_3["fea_1"] = 40*0.03
df_train_4["fea_1"] = 50*0.03
df_test_1["fea_1"] = 10*0.03
df_test_2["fea_1"] = 10*0.03
df_test_3["fea_1"] = 40*0.03
df_test_4["fea_1"] = 50*0.03

df_train = pd.concat([df_train_1,df_train_2,df_train_3,df_train_4],ignore_index=True)
df_test = pd.concat([df_test_1,df_test_2,df_test_3,df_test_4],ignore_index=True)
print(df_train.shape)
print(df_test.shape)
print(df_train.head(5))
print(df_test.head(5))

(183093, 11)
(46571, 10)
                   时间  辐照度        风速   风向        温度        压强        湿度  \
0 2016-04-01 00:15:00 -1.0 -0.707547  251 -0.090909 -0.030303 -0.157895   
1 2016-04-01 00:15:00 -1.0 -0.707547  251 -0.090909 -0.030303 -0.157895   
2 2016-04-01 00:30:00 -1.0 -0.716981  250 -0.107071 -0.030303 -0.136842   
3 2016-04-01 00:45:00 -1.0 -0.726415  248 -0.123232  0.030303 -0.094737   
4 2016-04-01 01:00:00 -1.0 -0.735849  244 -0.135354  0.030303 -0.073684   

   实发辐照度      实际功率  type  fea_1  
0    0.0 -0.019333     1    0.3  
1    0.0 -0.019333     1    0.3  
2    0.0 -0.021000     1    0.3  
3    0.0 -0.022000     1    0.3  
4    0.0 -0.022000     1    0.3  
   id                      时间  辐照度        风速   风向        温度        压强  \
0   1 2018-04-30 23:59:59.985 -1.0 -0.801887    8 -0.046465  0.272727   
1   2 2018-05-01 00:14:59.985 -1.0 -0.783019    2 -0.058586  0.272727   
2   3 2018-05-01 00:29:59.985 -1.0 -0.783019  358 -0.066667  0.272727   
3   4 2018-05-01 00:44:59.98

In [2]:
def time_extra(x):
    tmp = x.split(" ")
    tmp1 = tmp[0].split("-")
    tmp2 = tmp[1].split(":")
    year,month,day = int(tmp1[0]),int(tmp1[1]),int(tmp1[2])
    hour,minute = int(tmp2[0]),int(tmp2[1])
#     print(year,month,day,hour,minute)
    return year,month,day,hour,minute

def gen_fea(df):
    df["year"] = df["时间"].dt.year
    df["month"] = df["时间"].dt.month
    df["day"] = df["时间"].dt.day
    df["hour"] = df["时间"].dt.hour
#     df["xx1"] = df["辐照度"] - df["湿度"]
#     df["xx2"] = df["温度"] - df["湿度"]
#     df["xx3"] = df["辐照度"] + df["温度"]
#     df["hour_2"] = df["hour"].apply(lambda x:x%6)
#     df["hour_3"] = df["hour"].apply(lambda x:(x%12) - 7)
#     df["hour_4"] = df["hour"].apply(lambda x:13 - (x%12))
    df["minute"] = df["时间"].dt.minute
#     df["dow"] = df["时间"].dt.weekday
    df["minute"] = df["minute"].apply(lambda x: x + 1 if x in [59, 14, 29, 44] else x )
    return df

import datetime

def add_poly_features(data, column_names):
    features = data[column_names]
    rest_features = data.drop(column_names, axis=1)
    poly_transformer = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
    poly_features = pd.DataFrame(poly_transformer.fit_transform(features),
                                 columns=poly_transformer.get_feature_names(column_names))

    for col in poly_features.columns:
        rest_features.insert(1, col, poly_features[col])
    return rest_features


def get_sta_feat(data):
    data_temp = data.copy()
    data = data.sort_values(by=["type","时间"])
    data["diff_light"] = data["辐照度"].diff()
    data["diff_temp"] = data["温度"].diff()
    data["diff_wet"] = data["湿度"].diff()

#     merge_col = ["type","day"]
#     result = data_temp.groupby(merge_col, as_index=False)['辐照度'].agg({'day_light_min': 'min',
#                                                                              'day_light_mean': 'mean',
#                                                                              'day_light_median': 'median',
#                                                                              'day_light_max': 'max',
#                                                                              'day_light_std': 'std',
#                                                                              'day_light_skew': 'skew'
#                                                                             })
#     result = result.fillna(0)
# #     print(result.head(5))
#     data = pd.merge(data,result,on=merge_col,how="left")
    
#     merge_col = ["type","month"]
#     result = data_temp.groupby(merge_col, as_index=False)['辐照度'].agg({'month_light_min': 'min',
#                                                                              'month_light_mean': 'mean',
#                                                                              'month_light_median': 'median',
#                                                                              'month_light_max': 'max',
#                                                                              'month_light_std': 'std',
#                                                                              'month_light_skew': 'skew'
#                                                                             })
#     result = result.fillna(0)
# #     print(result.head(5))
#     data = pd.merge(data,result,on=merge_col,how="left")
    
    
#     merge_col = ["type","hour"]
#     result = data_temp.groupby(merge_col, as_index=False)['辐照度'].agg({'hour_light_min': 'min',
#                                                                              'hour_light_mean': 'mean',
#                                                                              'hour_light_median': 'median',
#                                                                              'hour_light_max': 'max',
#                                                                              'hour_light_std': 'std',
#                                                                              'hour_light_skew': 'skew'
#                                                                             })
#     result = result.fillna(0)
# #     print(result.head(5))
#     data = pd.merge(data,result,on=merge_col,how="left")
    
    merge_col = ["type","month","day"]
    result = data_temp.groupby(merge_col, as_index=False)['辐照度'].agg({
#                                                                             'month_day_light_min': 'min',
#                                                                              'month_day_light_mean': 'mean',
#                                                                              'month_day_light_median': 'median',
#                                                                              'month_day_light_max': 'max',
                                                                             'month_day_light_std': 'std',
#                                                                              'month_day_light_skew': 'skew'
                                                                            })
    result = result.fillna(0)
#     print(result.head(5))
    data = pd.merge(data,result,on=merge_col,how="left")
    
    merge_col = ["type","month","day"]
    result = data_temp.groupby(merge_col)['温度'].apply(lambda x: x.max()-x.min()).reset_index()
    result.columns=["type","month","day","max_sub_min_temp"]
    result = result.fillna(0)
#     print(result.head(5))
    data = pd.merge(data,result,on=merge_col,how="left")
    
#     merge_col = ["type","month","day"]
#     result = data_temp.groupby(merge_col)['湿度'].apply(lambda x: x.max()-x.min()).reset_index()
#     result.columns=["type","month","day","max_sub_min_wet"]
#     result = result.fillna(0)
# #     print(result.head(5))
#     data = pd.merge(data,result,on=merge_col,how="left")
    
    
    merge_col = ["type","month","day"]
    result = data_temp.groupby(merge_col, as_index=False)['温度'].agg({
#                                                                             'month_day_light_min': 'min',
#                                                                              'month_day_light_mean': 'mean',
#                                                                              'month_day_light_median': 'median',
#                                                                              'month_day_light_max': 'max',
                                                                             'month_day_temp_std': 'std',
#                                                                              'month_day_light_skew': 'skew'
                                                                            })
    result = result.fillna(0)
#     print(result.head(5))
    data = pd.merge(data,result,on=merge_col,how="left")
    

    merge_col = ["type","month","day"]
    result = data_temp.groupby(merge_col, as_index=False)['湿度'].agg({
#                                                                             'month_day_light_min': 'min',
#                                                                              'month_day_light_mean': 'mean',
#                                                                              'month_day_light_median': 'median',
#                                                                              'month_day_light_max': 'max',
                                                                             'month_day_wet_std': 'std',
#                                                                              'month_day_light_skew': 'skew'
                                                                            })
    result = result.fillna(0)
#     print(result.head(5))
    data = pd.merge(data,result,on=merge_col,how="left")
    
#     merge_col = ["type","month","day"]
#     result = data_temp.groupby(merge_col, as_index=False)['风速'].agg({
# #                                                                             'month_day_light_min': 'min',
# #                                                                              'month_day_light_mean': 'mean',
# #                                                                              'month_day_light_median': 'median',
# #                                                                              'month_day_light_max': 'max',
#                                                                              'month_day_wind_std': 'std',
# #                                                                              'month_day_light_skew': 'skew'
#                                                                             })
#     result = result.fillna(0)
# #     print(result.head(5))
#     data = pd.merge(data,result,on=merge_col,how="left")
    
#     merge_col = ["type","month","day"]
#     result = data_temp.groupby(merge_col, as_index=False)['压强'].agg({
# #                                                                             'month_day_light_min': 'min',
# #                                                                              'month_day_light_mean': 'mean',
# #                                                                              'month_day_light_median': 'median',
# #                                                                              'month_day_light_max': 'max',
#                                                                              'month_day_p_std': 'std',
# #                                                                              'month_day_light_skew': 'skew'
#                                                                             })
#     result = result.fillna(0)
# #     print(result.head(5))
#     data = pd.merge(data,result,on=merge_col,how="left")
    
    
    return data

def get_sta_feat_2(data,df_test):
    data_temp = data.copy()
    merge_col = ["type","day"]
    result = data_temp.groupby(merge_col, as_index=False)['实际功率'].agg({'day_true_light_min': 'min',
                                                                    'day_true_light_mean': 'mean',
                                                                    'day_true_light_median': 'median',
                                                                    'day_true_light_max': 'max',
                                                                    'day_true_light_std': 'std',
                                                                    'day_true_light_skew': 'skew'
                                                                    })
    result = result.fillna(0)
#     print(result.head(5))
    data = pd.merge(data, result, on=merge_col, how="left")
    df_test = pd.merge(df_test,result,on=merge_col,how="left")
    
    merge_col = ["type","hour"]
    result = data_temp.groupby(merge_col, as_index=False)['实际功率'].agg({'hour_true_light_min': 'min',
                                                                      'hour_true_light_mean': 'mean',
                                                                      'hour_true_light_median': 'median',
                                                                      'hour_true_light_max': 'max',
                                                                      'hour_true_light_std': 'std',
                                                                      'hour_true_light_skew': 'skew'
                                                                      })
    result = result.fillna(0)
#     print(result.head(5))
    data = pd.merge(data, result, on=merge_col, how="left")
    df_test = pd.merge(df_test, result, on=merge_col, how="left")
    
    merge_col = ["type","day","hour"]
    result = data_temp.groupby(merge_col, as_index=False)['实际功率'].agg({'day_hour_true_light_min': 'min',
                                                                      'day_hour_true_light_mean': 'mean',
                                                                      'day_hour_true_light_median': 'median',
                                                                      'day_hour_true_light_max': 'max',
                                                                      'day_hour_true_light_std': 'std',
                                                                      'day_hour_true_light_skew': 'skew'
                                                                      })
    result = result.fillna(0)
#     print(result.head(5))
    data = pd.merge(data, result, on=merge_col, how="left")
    df_test = pd.merge(df_test, result, on=merge_col, how="left")
    

    return data,df_test



df_train = gen_fea(df_train)
df_test = gen_fea(df_test)
df_train = get_sta_feat(df_train)
df_test = get_sta_feat(df_test)
df_train,df_test = get_sta_feat_2(df_train,df_test)


test_id = df_test[['id']]
col= [ i for i in df_test.columns if i in df_train.columns ]
col = [ i for i in col if i not in ["时间"]]
print(col)


['辐照度', '风速', '风向', '温度', '压强', '湿度', 'type', 'fea_1', 'year', 'month', 'day', 'hour', 'minute', 'diff_light', 'diff_temp', 'diff_wet', 'month_day_light_std', 'max_sub_min_temp', 'month_day_temp_std', 'month_day_wet_std', 'day_true_light_min', 'day_true_light_mean', 'day_true_light_median', 'day_true_light_max', 'day_true_light_std', 'day_true_light_skew', 'hour_true_light_min', 'hour_true_light_mean', 'hour_true_light_median', 'hour_true_light_max', 'hour_true_light_std', 'hour_true_light_skew', 'day_hour_true_light_min', 'day_hour_true_light_mean', 'day_hour_true_light_median', 'day_hour_true_light_max', 'day_hour_true_light_std', 'day_hour_true_light_skew']


In [3]:
# import xgboost as xgb
# train_B_flag = df_train['实际功率']
# dtrain_B = xgb.DMatrix(data=df_train[col],label=train_B_flag)


In [4]:

# Trate=0.25  
# params = {'booster':'gbtree',
#                   'eta': 0.1,
#                    'max_depth': 4,
#                    'max_delta_step': 0,
#                    'subsample':0.9,      
#                    'colsample_bytree': 0.9,
#                    'base_score': Trate,
#                    'objective': "reg:linear",
#                    'lambda':5,
#                    'alpha':8,
#                    'random_seed':100,
#                     "silent": 1
#                    }
# params['eval_metric'] = 'rmse' 
# xgb_model = xgb.train(params,dtrain_B,num_boost_round=500,maximize=True,verbose_eval=0)  

In [5]:
# from xgboost import plot_importance
# import matplotlib.pyplot as plt
# plt.rcParams['font.sans-serif']=['SimHei']
# plt.rcParams['axes.unicode_minus'] = False
# plt.rcParams['savefig.dpi'] = 300 #图片像素
# plt.rcParams['figure.dpi'] = 300 #分辨率
# plot_importance(xgb_model)
# plt.show()

In [6]:
# from sklearn.metrics import mean_squared_error
# train_pred = xgb_model.predict(xgb.DMatrix(df_train[col].fillna(-999))) 
# score = mean_squared_error(train_B_flag,train_pred)
# print(score)
# sub = df_test[["id"]]
# res = xgb_model.predict(xgb.DMatrix(df_test[col].fillna(-999))) 
# sub['predicition'] = res
# sub.to_csv('../submit/sub_baseline_%s.csv'%str(score),index = None)
# # sub.to_csv('../submit/sub_baseline_4.csv',index = None)

In [7]:
# 7.776276588388384
# 5.807599225474515
# 5.035979671146159   0.139182 
# 5.027925458771837   0.13842

# 5.025856517310198  0.13817

In [8]:
import lightgbm as lgb

train_matrix = lgb.Dataset(df_train[col], label=df_train['实际功率'])


params = {
        'learning_rate': 0.02,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'sub_feature': 0.7,
        'num_leaves': 200,
        'min_data': 100,
        'max_depth':6,
        'min_hessian': 1,
        'verbose': -1,
        'seed': 666,
        'nthread': 50,
    }



num_round = 2000
early_stopping_rounds = 50

model = lgb.train(params, train_matrix,num_round,valid_sets=train_matrix,
                  early_stopping_rounds=early_stopping_rounds,verbose_eval=200
                  )

Training until validation scores don't improve for 50 rounds.
[200]	training's rmse: 2.79203
[400]	training's rmse: 2.56741
[600]	training's rmse: 2.42574
[800]	training's rmse: 2.30944
[1000]	training's rmse: 2.21126
[1200]	training's rmse: 2.12085
[1400]	training's rmse: 2.04738
[1600]	training's rmse: 1.99198
[1800]	training's rmse: 1.93412
[2000]	training's rmse: 1.88553
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 1.88553


In [9]:
sub = df_test[["id"]]
res = model.predict(df_test[col].fillna(-999)) 
sub['predicition'] = res
sub.to_csv('../submit/lgb_baseline.csv',index = None)

# [2000]	training's rmse: 1.88553   

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
